Read listings, reviews and calendar

In [1]:
import pandas as pd
import locale

locale.setlocale(locale.LC_ALL, 'en_US')

# listings
df = pd.read_csv("../data/listings.csv.gz", compression="gzip")

# reviews and review languages
reviews = pd.read_csv("../data/reviews.csv.gz", compression="gzip")
reviews["date"] = pd.to_datetime(reviews["date"])
review_languages = pd.read_csv("../data/review_languages.csv.gz", compression="gzip")

# calendar with prices
calendar = pd.read_csv("../data/calendar.csv.gz", compression="gzip")
calendar["date"] = pd.to_datetime(calendar["date"])
calendar["price_numeric"] = calendar["price"].str[1:].map(locale.atof)

/var/folders/k6/hdzmrkf915d0twbr_qmhy9l00000gn/T/ipykernel_91469/2405712681.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  calendar = pd.read_csv("../data/calendar.csv.gz", compression="gzip")


Read parish definitions

In [2]:
import geopandas
from shapely.geometry import Point

df_parishes = geopandas.read_file("../data/lisbon_parishes.geojson")
df["geometry"] = df.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)

df_listings = geopandas.GeoDataFrame(df, crs="EPSG:4326", geometry="geometry")

Merge dataframes

In [3]:
merged_df = pd.merge(reviews, calendar, on=['listing_id', 'date'], how='left')
merged_df["price_numeric"] = merged_df["price_numeric"].fillna(0)

In [4]:
merged_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,available,price,adjusted_price,minimum_nights,maximum_nights,price_numeric
0,6499,18879225,2014-09-02,17027029,Simone,"Ola Bruno,\r<br/>\r<br/>Tive um mes Fantástico...",NaN,NaN,NaN,NaN,NaN,0.0
1,6499,21074122,2014-10-11,7661611,Cláudio,Encontramos o apartamento de Bruno exatamente ...,NaN,NaN,NaN,NaN,NaN,0.0
2,6499,24704004,2015-01-02,20348870,Rodrigo,Estivemos em Lisboa por aproximadamente 03 (tr...,NaN,NaN,NaN,NaN,NaN,0.0
3,6499,37894945,2015-07-11,34762351,Blandine,Superbe quartier très proche du tram et du tra...,NaN,NaN,NaN,NaN,NaN,0.0
4,6499,41672462,2015-08-08,39438424,Mirjana,"Très bel appartement, bien situé et à proximit...",NaN,NaN,NaN,NaN,NaN,0.0


Map listings to each parish

In [5]:
from collections import defaultdict

listings = []
listings_per_parish = defaultdict(list)

for _, parish in df_parishes.iterrows():
    filtered_listings = df_listings[df_listings["geometry"].within(parish.geometry)]
    listings.append(filtered_listings)
    listings_per_parish[parish.id].extend(filtered_listings.id.tolist())

df_listings = pd.concat(listings)
df_listings = df_listings[df_listings["number_of_reviews"] > 0]

Find start and end date and the number of quarters:

In [6]:
start_date = reviews.date.min()
end_date = reviews.date.max()
quarters = reviews["date"].dt.to_period('Q').unique()

print(start_date)
print(end_date)
print(len(quarters))

2010-07-24 00:00:00
2024-12-15 00:00:00
58


We iterrate over each quarter and each parish. Then we create one tuple which includes the parish, the quarter, number of reviews and main language:

In [7]:
from tqdm import tqdm

merged_df["quarter"] = reviews['date'].dt.to_period('Q')
parish_languages = []

with tqdm(total=len(quarters) * len(listings_per_parish)) as pbar:
    # each quarter
    for name, groups in merged_df.groupby("quarter"):
    
        # each parish
        for parish, listing_ids in listings_per_parish.items():
            # get reviews for the respective parish
            rows = groups[groups["listing_id"].isin(listing_ids)]
            language_mapping = review_languages[review_languages["id"].isin(rows["id"])]
    
            # if there are reviews in the selected period, determine the "main" language
            if len(rows) > 0:
                language_mode = language_mapping.language.mode()
                parish_languages.append((
                    parish, name, len(rows), language_mode.iloc[0], rows["price_numeric"].mean()
                ))
            
            pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7714/7714 [00:51<00:00, 148.51it/s]


In [8]:
parish_languages[:10]

[(5328942886, Period('2010Q3', 'Q-DEC'), 3, 'en', np.float64(0.0)),
 (5328942886, Period('2010Q4', 'Q-DEC'), 1, 'en', np.float64(0.0)),
 (5328942886, Period('2011Q1', 'Q-DEC'), 1, 'en', np.float64(0.0)),
 (5553082417, Period('2011Q1', 'Q-DEC'), 2, 'en', np.float64(0.0)),
 (5309790438, Period('2011Q2', 'Q-DEC'), 1, 'en', np.float64(0.0)),
 (5328942886, Period('2011Q2', 'Q-DEC'), 2, 'en', np.float64(0.0)),
 (5309790438, Period('2011Q3', 'Q-DEC'), 1, 'en', np.float64(0.0)),
 (5328942886, Period('2011Q3', 'Q-DEC'), 1, 'en', np.float64(0.0)),
 (5527372459, Period('2011Q3', 'Q-DEC'), 9, 'en', np.float64(0.0)),
 (5553082417, Period('2011Q3', 'Q-DEC'), 1, 'en', np.float64(0.0))]

In [9]:
import csv

with open("../data/parish_data_quarterly.csv", "wt") as fp:
    writer = csv.writer(fp)
    writer.writerow(("parish_id", "quarter", "num_reviews", "language", "avg_price"))
    writer.writerows(parish_languages)